<a href="https://colab.research.google.com/github/mn-rahul/Amazon-ML/blob/main/Temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import csv
import pandas as pd

In [ ]:
!wget "https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip" 

--2021-07-31 07:42:30--  https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.124.162
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.124.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1061576029 (1012M) [binary/octet-stream]
Saving to: ‘dataset52a7b21.zip’

dataset52a7b21.zip  100%[===================>]   1012M  18.1MB/s    in 60s     

2021-07-31 07:43:31 (16.8 MB/s) - ‘dataset52a7b21.zip’ saved [1061576029/1061576029]



In [ ]:
!unzip dataset52a7b21.zip

Archive:  dataset52a7b21.zip
replace dataset/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dataset/train.csv       y
y
y

replace dataset/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: dataset/sample_submission.csv  
replace dataset/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: dataset/test.csv        
replace dataset/.~lock.train.csv#? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: dataset/.~lock.train.csv#  


In [ ]:
train=pd.read_csv('dataset/train.csv',escapechar = "\\",quoting = csv.QUOTE_NONE)

In [ ]:
X = train['TITLE'].astype('str')

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y=le.fit_transform(train['BROWSE_NODE_ID'])
y=np.expand_dims(y, axis=1)

In [ ]:
from tensorflow.keras.preprocessing import text, sequence
max_features = 200000
max_text_length = 400

x_tokenizer =text.Tokenizer(max_features)
x_tokenizer.fit_on_texts(list(X))
x_tokenized = x_tokenizer.texts_to_sequences(X)
x_train_val = sequence.pad_sequences(x_tokenized, maxlen=max_text_length)

In [ ]:
# x_tokenizer.word_index

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-07-31 07:50:46--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-07-31 07:50:46--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-07-31 07:50:46--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
!unzip -q glove.6B.zip

In [ ]:
embedding_dim = 300
embedding_index = dict()
f = open('glove.6B.300d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embedding_index))

Loaded 400000 word vectors.


In [ ]:
!rm glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip dataset52a7b21.zip  glove.6B.200d.txt  glove.6B.50d.txt

In [ ]:
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, index in x_tokenizer.word_index.items():
  if index>max_features -1:
    break
  else:
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[index] = embedding_vector
del(embedding_vector)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam#Try all of these
from sklearn.preprocessing import LabelEncoder


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam#Try all of these
model = Sequential()
model.add(Embedding(max_features,
                    embedding_dim,
                    embeddings_initializer=tf.keras.initializers.Constant(
                        embedding_matrix),trainable=False))
model.add(Dropout(0.2))

In [ ]:
filters = 250
kernel_size = 3
hidden_dims = 25000
no_of_classes=len(train['BROWSE_NODE_ID'].unique())
no_of_classes

9919

In [ ]:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid'))
model.add(MaxPooling1D())
model.add(Conv1D(filters,
                 5,
                 padding='valid',
                 activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_dims, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(no_of_classes,activation='softmax'))

model.summary()

In [ ]:
batch_size = 16
epochs = 10

#callbacks = [EarlyStopping(monitor='val_loss')]
hist_adam = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_val,y_val))
#        callbacks=callbacks) 
model = Model(inputs, output)

opt = Adam(lr=1e-3) model.compile(loss='categorical_crossentropy',optimizer=opt)

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss')] hist_adam = model.fit(X_train, y_train, batch_size=1000, epochs=20, verbose=2, validation_data=(X_val, y_val), callbacks=callbacks

In [ ]:
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.xlabel('Epoch', fontsize=14)
plt.plot(hist_adam.history['loss'], color='b', label='Training Loss')
plt.plot(hist_adam.history['val_loss'], color='r', label='Validation Loss')
plt.legend(loc='upper right')